In [9]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm
import metnum
import numpy
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import csv
import time


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def correr_Knn_con_k_aumentando_en(porcentage_para_entrenar,cant_muestras=42000,semilla = 2,intervalo_k):
    df_train = pd.read_csv("../data/train.csv")
    df_train =  df_train.sample(frac=1, random_state=semilla)
    X= df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)

    accuracy_of_all_k = []
    precision_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    recall_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    f1_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    times = [] 
    
    setup_time_start_time = time.time()
    
    limit = int(0.8 * X.shape[0]) 
    X_train = X[:limit]
    X_val = X[limit:]
    y_train = y[:limit]
    y_val = y[limit:]
    alpha = 0
    # Hago el fit generico que guarda los datos 
    clf.fit(X_train, y_train)
    setup_time_end_time = time.time()
    setup_time = setup_time_end_time-setup_time_start_time
    for k in tqdm(range(1,X_train.shape[0]+1,intervalo_k)):

        # Correr knn 
        clf = metnum.KNNClassifier(k)
        knn_start_time = time.time()
   
        y_pred = clf.predict(X_val)
        print(accuracy_score(y_val, y_pred))
        knn_end_time = time.time()
        knn_time = knn_end_time-knn_start_time
        # Calcular metricas de interes
        labels= [0,1,2,3,4,5,6,7,8,9]
        precision = precision_score(y_val, y_pred,labels=labels, average=None)
        accuracy =  accuracy_score(y_val, y_pred)
        recall   =  recall_score(y_val,y_pred,labels=labels, average=None)
        f1 = f1_score(y_val,y_pred,labels=labels, average=None)

        # Escribir los resultados
        accuracy_of_all_k.append([alpha, k ,accuracy])
        times.append(setup_time+knn_time)

            # Agregar una fila al dataframe de precision
        digit = 0 
        precision_dict={}
        for i in range(0,10,1):
            precision_dict[digit]=precision[i]
            digit += 1 
        precision_dict['k']=k
        precision_dict['alpha']=alpha
        precision_df = precision_df.append(precision_dict,ignore_index=True)

             # Agregar una fila al dataframe de recall
        digit = 0 
        recall_dict={}
        for i in range(0,10,1):
            recall_dict[digit]=recall[i]
            digit += 1 
        recall_dict['k']=k
        recall_dict['alpha']=alpha
        recall_df = recall_df.append(recall_dict,ignore_index=True)
        
            # Agregar una fila al dataframe de f1
            
        digit = 0 
        f1_dict={}
        for i in range(0,10,1):
            f1_dict[digit]=f1[i]
            digit += 1 
        f1_dict['k']=k
        f1_dict['alpha']=alpha
        f1_df = f1_df.append(f1_dict,ignore_index=True)
            

    # Escribo los resultados a un archivo para no tener que correr devuelta los resultados.
    precision_df.to_csv('knn_solo_precision.csv', index=False) 
    recall_df.to_csv('knn_solo_recall.csv', index=False)   
    f1_df.to_csv('knn_solo_f1.csv', index=False)   

    with open('knn_solo_acuracy.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(accuracy_of_all_k)

    with open('knn_solo_time.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(times)
        
    
    with open('knn_solo_predicciones.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(y_pred)



    

SyntaxError: non-default argument follows default argument (<ipython-input-10-e27f69cc1017>, line 1)

In [2]:
correr_Knn_con_todos_los_k_posibles(0.8)

NameError: name 'correr_Knn_con_todos_los_k_posibles' is not defined

In [11]:
def correr_Knn_pca(porcentage_para_entrenar,cant_muestras=42000,semilla = 2,intervalo_k,intervalo_alpha):
    df_train = pd.read_csv("../data/train.csv")
    df_train =  df_train.sample(frac=1, random_state=semilla)
    X_original = df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)

    valores_k = [1,5,10,25,50,75,100,200,500,1000,2000,5000,10000,20000]
    accuracy_of_all_k = []
    precision_df = pd.DataFrame(columns=('alpha','k',0,1,2,3,4,5,6,7,8,9))
    recall_df = pd.DataFrame(columns=('alpha','k',0,1,2,3,4,5,6,7,8,9))
    f1_df = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9))
    times = [] 
    for alpha in tqdm(range(1,29,intervalo_alpha)):
        
        setup_time_start_time = time.time()
        pca = metnum.PCA(alpha)
        X = pca.transform(X_original)
        limit = int(0.8 * X.shape[0]) 
        X_train = X[:limit]
        X_val = X[limit:]
        y_train = y[:limit]
        y_val = y[limit:]

        
        # Hago el fit generico que guarda los datos 

        setup_time_end_time = time.time()
        setup_time = setup_time_end_time-setup_time_start_time
        
        for k in tqdm(valores_k):

            # Correr knn 
            clf = metnum.KNNClassifier(k)
  
            knn_start_time = time.time()
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_val)
            print(accuracy_score(y_val, y_pred))
            knn_end_time = time.time()
            knn_time = knn_end_time-knn_start_time
            # Calcular metricas de interes
            labels= [0,1,2,3,4,5,6,7,8,9]
            precision = precision_score(y_val, y_pred,labels=labels, average=None)
            accuracy =  accuracy_score(y_val, y_pred)
            recall   =  recall_score(y_val,y_pred,labels=labels, average=None)
            f1 = f1_score(y_val,y_pred,labels=labels, average=None)

            # Escribir los resultados
            accuracy_of_all_k.append([alpha, k ,accuracy])
            times.append(setup_time+knn_time)

                # Agregar una fila al dataframe de precision
            digit = 0 
            precision_dict={}
            for i in range(0,10,1):
                precision_dict[digit]=precision[i]
                digit += 1 
            precision_dict['k']=k
            precision_dict['alpha']=alpha
            precision_df = precision_df.append(precision_dict,ignore_index=True)

                 # Agregar una fila al dataframe de recall
            digit = 0 
            recall_dict={}
            for i in range(0,10,1):
                recall_dict[digit]=recall[i]
                digit += 1 
            recall_dict['k']=k
            recall_dict['alpha']=alpha
            recall_df = recall_df.append(recall_dict,ignore_index=True)
            
                    
            # Agregar una fila al dataframe de f1
            
            digit = 0 
            f1_dict={}
            for i in range(0,10,1):
                f1_dict[digit]=f1[i]
                digit += 1 
            f1_dict['k']=k
            f1_dict['alpha']=alpha
            f1_df = f1_df.append(f1_dict,ignore_index=True)


        # Escribo los resultados a un archivo para no tener que correr devuelta los resultados.
        precision_df.to_csv('knn_pca_precision_re_do.csv', index=False) 
        recall_df.to_csv('knn_pca_recall.csv_re_do', index=False)   

        with open('knn_pca_acuracy_re_do.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(accuracy_of_all_k)

        with open('knn_pca_time_re_do.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(times)

        with open('knn_pca_predicciones.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(y_pred)

 

    

SyntaxError: non-default argument follows default argument (<ipython-input-11-ca8ba29e5871>, line 1)

In [12]:
correr_Knn_pca(0.8)

NameError: name 'correr_Knn_pca' is not defined